<a href="https://colab.research.google.com/github/ehennis/Manning/blob/master/Milestone6_Answers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Deploying a trained network  

This will be slightly different from the first 5 milestones in that we are targeting JavaScript and the web.  
* The first section will be using Python to convert the DataFrame into a JS file.  
* The second part will be using Python to create an h5 model.  
* The final part will be adding the HTML and JavaScript code around the model.

In [0]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras

#Load the data set with all of the calculated values
column_names = ['Date','HomeTeam','HomeScore','AwayTeam','AwayScore',
                'HomeScoreAverage','HomeDefenseAverage','AwayScoreAverage','AwayDefenseAverage',
                'Result']

games_csv = 'https://liveproject-resources.s3.amazonaws.com/other/deeplearningbasketballscores/Games-Calculated.csv'
all_data = pd.read_csv(games_csv, header=None, names=column_names)

#Load the teams data set
team_columns = ['Conference','Team']

teams_csv = 'https://liveproject-resources.s3.amazonaws.com/other/deeplearningbasketballscores/Teams.csv'
teams = pd.read_csv(teams_csv, header=None, names=team_columns)

## Create a JavaScript Array  

In this section, we are going to grab the values from the current season (as stated in the assignment, this would normally be done during the season and incremented each day but for this project we will just use the final stats) and export those to a text file.

In [0]:
# Find the averages
def FindAverages(df_current, team_name, col_name, off_avg, def_avg):
  row = df_current[(df_current[col_name] == team_name)][-1:]
  #If the team can't be found return 0s
  if row.empty:
      return 0,0
  return row[off_avg].values[0], row[def_avg].values[0]
  
def FindHomeAverages(df, team_name):
  return FindAverages(df, team_name,'HomeTeam','HomeScoreAverage','HomeDefenseAverage')

def FindAwayAverages(df, team_name):
  return FindAverages(df, team_name,'AwayTeam','AwayScoreAverage','AwayDefenseAverage')

def PrintCurrentStats(file1, df, teams):
  #Grab each team and find their home and away stats
  file1.write('var current_stats = [\n')
  for index, row in teams.iterrows():
      team_name = row['Team']
      home_off, home_def = FindHomeAverages(df, team_name)
      away_off, away_def = FindAwayAverages(df, team_name)
      # Fix the Quotes
      if( "'" in team_name):
        team_name = team_name.replace("\'","\\'");
      #Print them so that I can use their values in a JavaScript file
      file1.write('[ \'%s\',%s,%s,%s,%s ],\n' % (team_name,home_off, home_def, away_off, away_def))
  file1.write('];')

def PrintConferenceAndTeams(file1):
  file1.write("var conf = ['Select Conference','America East','American Athletic Conference','Atlantic 10','Atlantic Coast','Atlantic Sun','Big 12','Big East','Big Sky',");
  file1.write("'Big South','Big Ten','Big West','Colonial Athletic','Conference USA','Horizon','Ivy League','Metro Atlantic Athletic','Mid-American','Mid-Eastern',");
  file1.write("'Missouri Valley','Mountain West','Northeast','Ohio Valley','Pac-12','Patriot League','Southeastern','Southern','Southland','Southwestern Athletic',");
  file1.write("'Summit League','Sun Belt','West Coast','Western Athletic'];");
  file1.write('\n');

  file1.write("//Teams teamDictionary\n")
  file1.write("var teamDict = new Object();\n")
  for index, row in teams.iterrows():
    team_name = row['Team']
    # Fix the Quotes
    if( "'" in team_name):
      team_name = team_name.replace("\'","\\'");
    conference = row['Conference']
    file1.write("teamDict['%s'] = '%s';\n" % (team_name,conference));


In [0]:
# Create the file
file1 = open('Teams.js','w')

# Write Conference and Teams for the dropdowns
PrintConferenceAndTeams(file1);

#Grab Current Season
s2018 = all_data[(all_data['Date'] > '2018-11-01') & (all_data['Date'] < '2019-04-15')].copy()
PrintCurrentStats(file1, s2018, teams)


file1.close()

In [0]:
#Google Colab code to download
from google.colab import files
files.download('Teams.js')

## Creating and deploying a trained model  

### Convert from Keras to TF.JS

This section we will take our trained model from milestone 5 and convert it to a format that our web site can consume.

In [34]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# Load h5

# TODO: Fix this once it gets uploaded
model = keras.models.load_model('/content/gdrive/My Drive/Colab Notebooks/deeplearning-manning.h5')

# Recompile Model
opt = keras.optimizers.RMSprop()
m = [
      keras.metrics.MeanAbsoluteError(),
      keras.metrics.Accuracy(),
      keras.metrics.MeanSquaredError()
]
l = keras.losses.MeanSquaredError()

model.compile(loss=l, optimizer=opt, metrics=m)

In [0]:
# Need to install TensorFlow.JS
!pip install tensorflowjs

import tensorflowjs as tfjs
print(tfjs.__version__)

In [42]:
tfjs.converters.save_keras_model(model,'dl-manning')


/usr/local/lib/python3.6/dist-packages/tensorflowjs/converters/keras_h5_conversion.py:122: H5pyDeprecationWarning: The default file mode will change to 'r' (read-only) in h5py 3.0. To suppress this warning, pass the mode you need to h5py.File(), or set the global default h5.get_config().default_file_mode, or set the environment variable H5PY_DEFAULT_READONLY=1. Available modes are: 'r', 'r+', 'w', 'w-'/'x', 'a'. See the docs for details.
  return h5py.File(h5file)


In [0]:
#Google Colab code to download
from google.colab import files
files.download('dl-manning/model.json')
files.download('dl-manning/group1-shard1of1.bin')

### Create the JavaScript files to access the model  

#TODO: Figure out how to handle this. Currently, WebSite.zip has the files